In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

# Natural Language Processing
<!-- requirement: small_data/apple_fruit.txt -->
<!-- requirement: small_data/apple_inc.txt -->
<!-- requirement: small_data/ford_car.txt -->
<!-- requirement: small_data/ford_crossing.txt -->
<!-- requirement: small_data/window_glass.txt -->
<!-- requirement: small_data/windows_ms.txt -->


Natural language processing (NLP) is concerned with developing algorithms that enable computers to process or understand human (or "natural") language. In this notebook we will focus on using natural language processing to classify texts into different genres. Natural language processing is also concerned with language generation and translation, but these topics are rather advanced and are beyond the scope of this notebook. 

**EXAMPLE:**
In a given source block of (prose) text, you may want to be able to tell apart Apple (the company) vs. apple (the fruit).  Ideally you would also be able to tell apart Ford vs ford, Windows vs windows, etc. via similar examples.

**The type of learner**: We're going to choose to look at this as a _supervised classification_ problem.  There are also unsupervised approaches, but you have to make choices sometimes.  This means we need some "marked up" data:

> **The training data set**: Having limited resources at our disposal, we're going to try to use Wikipedia's articles on the given topics as our chosen "corpus" of text.

> **The test data set**: A good idea would be to mark up a small corpus by hand, or to use sentences culled from Wikipedia with the disambiguation coming from looking at the target of outgoing links.  We're not going to be that scientific for lack of time.

## Text as a "bag of words"


You can imagine that documents belonging to the same class will share more words in common with each other than with documents belonging to different classes. This idea is key to how we will construct our classifier. As such, as a first step, we will keep track of the word counts (or frequencies) for each document. 

  - Split the text into words
  - Count how many times each word (in some fixed vocabulary) occurs
  - _(Optionally)_ normalize the counts against some baseline
  - _(Variant)_ Just do a binary "yes / no" for whether each word (in some vocabulary) is contained in the material
  
Let us count the words we see in documents (in this case, sentences) pertaining to apples (the fruit) and Apple (the company). Our first step will be to pull in the training (and test) data. We will want to clean both data on the way in: our goal is to have each text as a list of strings, one string for each sentence. We'll be using `spaCy` for this. It should already be installed, and its data set downloaded, on your box.

In [ ]:
import spacy
nlp = spacy.load("en")

When `spaCy` loads a document, it automatically tokenizes it into sentences and words. For example:

In [ ]:
doc = nlp('Here is a text document. spaCy requires it to be a unicode string.')
# doc.sents is a generator producing sentences
for sentence in doc.sents:
    print(sentence)
# doc can be indexed to find the individual words
print("Word 3:", doc[3])

Now let's grab our documents from the "apple" and "Apple" Wikipedia pages.

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

# Spit out (slightly cleaned up) sentences from a Wikipedia article.
def wikipedia_to_sents(url):
    soup = BeautifulSoup(urlopen(url), 'lxml').find(attrs={'id':'mw-content-text'})
    
    # The text is littered by references like [n].  Drop them.
    def drop_refs(s):
        return ''.join(re.split('\[\d+\]', s))
    
    paragraphs = [drop_refs(p.text) for p in soup.find_all('p')]
    return [s.text for paragraph in paragraphs for s in nlp(paragraph).sents if len(s) > 2]

fruit_sents = wikipedia_to_sents("http://en.wikipedia.org/wiki/Apple")
company_sents = wikipedia_to_sents("http://en.wikipedia.org/wiki/Apple_Inc.")

In [ ]:
company_sents[-105:-100]

In [ ]:
fruit_sents[-105:-100]

### Count Vectorizer


Machine learning algorithms, however, prefer vectors of numbers, not text. When we do this translation, the output will be typically be a very large, but usually sparse, vector: The number of coordinates is the number of words in our dictionary, and the $i$-th coordinate entry is the number of occurrences of the $i$-th word.

There's a reasonable implementation of this in the `CountVectorizer` class in `sklearn.feature_extraction.text`. See http://scikit-learn.org/stable/modules/classes.html#text-feature-extraction-ref for more detail on the options. When we use sklearn's `CountVectorizer`, we generate a matrix where each column corresponds to a word, each row corresponds to a document, and the values are the word counts.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

bag_of_words_vectorizer = CountVectorizer()

counts = bag_of_words_vectorizer.fit_transform( fruit_sents + company_sents  )
print(counts.shape)

In [ ]:
# Note that counts is a **sparse** matrix.
print(counts.toarray())       #This is what it actually looks like.. there are non-zero entries, really!
print("\n", counts)           # .. this is just describing the non-zero entries

### Hashing Vectorizer


When doing "bag of words" type techniques on a *large* corpus and without an existing vocabulary, there is a simple trick that is often useful.  The issue (and solution) is as follows: 

 - The output is a feature vector, so that whenever we encounter a word we must look up which coordinate slot it is in.  A naive way would be to keep a list of all the words encountered so far, and look up each word when it is encountered.  Whenever we encounter a new word, we see if we've already seen it before and if not -- assign it a new number.  This requires storing all the words that we have seen in memory, cannot be done in parallel (because we'd have to share the table of seen words), etc.
 - A **hash function** takes as input something complicated (like a string) and spits out a number, with the desired property being that different inputs *usually* produce different outputs.  (This is how hash tables are implemented, as the name suggests.)
 - So -- rather than exactly looking up the coordinate of a given word, we can just use its hash value (modulo a big size that we choose).  This is fast and parallelizes easily.  (There are some downsides: You cannot tell, after the fact, what word each of your feature actually corresponds to!)
 
Scikit-learn includes `sklearn.feature_extraction.text.HashingVectorizer` to do this.  It behaves as almost a drop-in replacement for `CountVectorizer`.

## Word importance: Term frequency–inverse document frequency (TF-IDF)


Just using counts (or frequencies), as above, is clearly not great.  Both apples, the fruit, and Apple, the company, are enjoyed around the world!  We would like to find words that are common in one document, but not common in all of them.  This is the goal of the __tf-idf weighting__.  A precise definition is:


  1. If $d$ denotes a document and $t$ denotes a term, then the _raw term frequency_ $\mathrm{tf}^{raw}(t,d)$ is
  $$ \mathrm{tf}^{raw}(t,d) = \text{the number of times the term $t$ occurs in the document $d$} $$
  The vector of all term frequencies can optionally be _normalized_ either by dividing by the maximum of any single word's occurrence count ($L^1$) or by the Euclidean length of the vector of word occurrence counts ($L^2$).  Scikit-learn by default does this second one:
  $$ \mathrm{tf}(t,d) = \mathrm{tf}^{L^2}(t,d) = \frac{\mathrm{tf}^{raw}(t,d)}{\sqrt{\sum_t \mathrm{tf}^{raw}(t,d)^2}} $$
  2. If $$ D = \left\{ d : d \in D \right\} $$ is the set of possible documents, then  the _inverse document frequency_ is
  $$ \mathrm{idf}^{naive}(t,D) = \log \frac{\# D}{\# \{d \in D : t \in d\}} \\
  = \log \frac{\text{count of all documents}}{\text{count of those documents containing the term $t$}} $$
  with a common variant being
  $$ \mathrm{idf}(t, D) = \log \frac{\# D}{1 + \# \{d \in D : t \in d\}} \\
   = \log \frac{\text{count of all documents}}{1 + \text{count of those documents containing the term $t$}} $$
  (This second one is the default in Scikit-learn. Without this tweak we would omit the $1+$ in the denominator and have to worry about dividing by zero if $t$ is not found in any documents.)
  3. Finally, the weight that we assign to the term $t$ appearing in document $d$ and depending on the corpus of all documents $D$ is
  $$ \mathrm{tfidf}(t,d,D) = \mathrm{tf}(t,d) \mathrm{idf}(t,D) $$
  

### Exercises:

  1. Imagine that $D$ consists of just two documents $D = \{ d_1, d_2 \}$ and that the word "cultivar" occurs in $d_1$ but not in $d_2$.  What is 
  $$ \mathrm{tfidf}(\mathrm{"cultivar"}, d_i, D)$$
for each $i=1,2$?  For simplicity, use $\mathrm{tf}^{raw}$ and the version of `idf` without the $1+$.  

  2. Same question as 1, but now use $\mathrm{tf}^{L^2}$ and the version of of `idf` with the $1+$.  

  3. What happens to the tf-idf weighting of a word if it occurs in all (or all but one) documents?  Consider both forms of `idf`.
  
  4. In the example below, we consider each sentence as a separate document for the purpose of tf-idf.  What happens if you instead treat the input as just two documents, one for each starting article.
  

### Hints/Answers:

  1. For $i=2$ it is zero.  For $i=1$, it is the number of occurrences of "cultivar" in $d_1$ multiplied by $\log 2$.
  2. For $i=2$, it is zero.  For $i=1$, it is .. also zero.
  3. Answer: If the word occurs in all documents, and there are $N$ of them, then the $1+$ form weights `idf` by $\log N/(N+1) < 0$ while the other form weights `idf` by $\log N/N = 0$.  If the word occurs in all-but-one document, then the $1+$ form weights `idf` by $0$ while the other form weights it by $\log N/(N-1) \approx 1+1/N$.
  4. It works less well, because of what's discussed in 3.  tf-idf doesn't work so well with few documents and where relevant words  occur (even if with wildly different frequencies!) in both.

### TF-IDF Vectorizer

The `CountVectorizer` and `HashingVectorizer` can be used to compute tf-idf values by combining them with the `TfidfTransformer` (the `TfidfVectorizer` is the `CountVectorizer` together with the `TfidfTransformer`). For our application (where the training and test data is small), we may as well just use `TfidfVectorizer` -- but it is good to know that `HashingVectorizer` is there.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from spacy.lang.en.stop_words import STOP_WORDS

ng_tfidf=TfidfVectorizer(max_features=300, 
                         ngram_range=(1,2), 
                         stop_words=STOP_WORDS.union({'apple', 'apples'}))
ng_tfidf=ng_tfidf.fit( fruit_sents + company_sents )
print(ng_tfidf.get_feature_names()[100:105])
print(ng_tfidf.transform(fruit_sents + company_sents))

## Document similarity metrics


A common problem is looking up a document similar to a given snippet, or relatedly comparing two documents for similarity.  The above provides a simple method for this called __cosine similarity__:
  - To each of the two documents $d_1, d_2$ in a corpus of documents $D$, assign its tf or tf-idf vector $$ (v_i)_{j} = \mathrm{tfidf}( t_{j}, d_i, D ) $$
  where $i$ ranges over indices for documents, and $j$ ranges over indices for terms in the vocabulary.
  - To compare two documents, simply find the cosine of the angle between the vectors:
  $$ \frac{v_i \cdot v_{i'}}{|v_i| |v_{i'}|} $$
  
(There's also a variant using binary vectors and Jaccard distance.)

## Engineering your features


There are many considerations to make when deciding which "words" or features to include in your classifier. In our example application, we might try to consider:
   - Capitalization of the word apple? (`_a_pple` vs `_A_pple`)    
   - Pluralization of the word apple? (apples)
   - Possessive form of the word apple? (Apple's)
   - Presence (or frequency) of certain well-chosen words : Does (e.g.,) the word "computer" or "fruit" occur in the sentence?  (This feature regards the sentence as a simple __bag of words__ without regard to trying to parse its structure.)
   - In addition to single words, we can also look for __n-grams__: Strings of n consecutive words.
   - Using the __tf-idf__ values of words or n-grams.

### Stop words

It's common to want to __omit__ certain common words when doing these counts -- "a", "an", and "the" are common enough so that their counts do not tend to give us any hints as to the meaning of documents.  Such words that we want to omit are called __stop words__ (they don't stop anything, though). NOTE: If you are using tf-idf and not the word frequency to compute your document similarity, however, you don't need to exclude stop words.

`spaCy` contains a standard list of such stop words for English in `spacy.lang.en.stop_words.STOP_WORDS`.  In our application, we'd also want to include "apple" -- it is certainly not going to help us distinguish our two meanings!

In [ ]:
STOP_WORDS

In [ ]:
# o_O
print('six' in STOP_WORDS, 'seven' in STOP_WORDS, 'eight' in STOP_WORDS)

In [ ]:
# The vocabulary *can* be built for you.  
#
# For instance, here we'll compute and then use the top 300 words by frequency -- *ignoring*
# the so-called "stopwords": these are words like "a", "and", "the" that are very common
# "apple" is not useful for distinguishing the two, but is common, so add it as a stopword.
#
# Nevertheless, this method is probably NOT GOOD.  See tf-idf instead.
counter=CountVectorizer(max_features=300,
                        stop_words=STOP_WORDS.union({'apple'}))
counter=counter.fit( fruit_sents + company_sents )
print(counter.get_feature_names())

# Now we can use it with that vectorizer, like so...
counter.transform(company_sents)
counter.transform(fruit_sents)

### Tokenization


Tokenization refers to splitting the text into pieces, in this case into sentences and into words. Instead of looking at just single words, it is also useful to look at **n-grams**: These are n-word long sequences of words (i.e., each of "farmer's market", "market share", and "farm share" is a 2-gram).

The exact same sort of counting techniques apply.  The `CountVectorizer` function has built in support for this, too:

If you pass it the `ngram_range=(m, M)` then it will count $n$-grams with  $m \leq n \leq M$.

In [ ]:
ng_counter=CountVectorizer(max_features=300, 
                           ngram_range=(2,2), 
                           stop_words=STOP_WORDS.union({'apple', 'Apple'}))
ng_counter=ng_counter.fit( fruit_sents + company_sents  )
print(ng_counter.get_feature_names())

# Now we can use it with that vectorizer, like so...
ng_counter.transform(company_sents)
ng_counter.transform(fruit_sents)

In [ ]:
# we have 300 "features"
print(len(ng_counter.get_feature_names()))

### Stemming


In our original hand-built vocabulary, we had to include both "apple" and "apples".  It would have been useful to identify them as one word.

This is not limited to just trailing "s" characters: e.g., the words "carry", "carries", "carrying", and "carried" all carry -- roughly -- the same meaning.  The process of replacing them by a common root, or **stem**, is called stemming -- the stem will not, in general, be a full word itself.

There's a related process called **lemmatization**: The analog of the "stem" here _is_ an actual word.  After `spaCy` processes some text, the `lemma_` property of each word contains the lemmatized version.

In [ ]:
print([w.lemma_ for w in nlp('carry carries carrying carried')])
print([w.lemma_ for w in nlp('eat eating eaten ate')])
print(' '.join(w.lemma_ for w in nlp("The quick brown fox jumped over the lazy dog.  "
                                     "I can't believe it's not butter.  "
                                     "I tried to ford the river and my unfortunate oxen died.")))

We can tell our bag-of-words counters (or tf-idf) to run on lemmatized text.  This way it won't have to include both e.g., 'apple' and 'apples'.  The way to do this with `CountVectorizer` or `TfidfVectorizer` is to supply a function with the `tokenizer` option.  This function can process the text before performing the counts (or computing tf-idf values).  

In [ ]:
def tokenize_lemma(text):
    return [w.lemma_ for w in nlp(text)]

stop_words_lemma = set(w.lemma_ for w in nlp(' '.join(STOP_WORDS)))

ng_stem_tfidf = TfidfVectorizer(max_features=300, 
                                ngram_range=(1,2), 
                                stop_words=stop_words_lemma.union({"apple"}),
                                tokenizer=tokenize_lemma)
ng_stem_tfidf = ng_stem_tfidf.fit(fruit_sents + company_sents)

ng_stem_vocab = ng_stem_tfidf.get_feature_names()
print(ng_stem_vocab)

### Part of speech tagging

Consider the "Ford" vs "ford" example.  As a human being, the easiest way to tell these apart is that "Ford" is a __noun__ while "ford" is a __verb__.

Fortunately, `spaCy` also has a part-of-speech tagger: You give it a sentence, and it tries to tag the parts of speech (e.g., noun, verb, adjective, etc.).  The broad category is given in the `.pos_` property, while a more detailed description, using the [UPenn Treebank Tags](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html), is in the `.tag_` property.

(N.B. Nothing's perfect -- the tagger will make mistakes.)

In [ ]:
s1 = "I tried to ford the river, and my unfortunate oxen died."
s2 = "Henry Ford built factories to facilitate the construction of the Ford automobile."

In [ ]:
[(w.text, w.pos_, w.tag_) for w in nlp(s1)]

In [ ]:
[(w.text, w.pos_, w.tag_) for w in nlp(s2)]

### Capitalization, punctuation, etc.

There are the obvious features that we had in mind....

## Building the Classifier

Disclaimer: This version is actually pretty bad -- it uses many of the right ideas, but puts them together pretty poorly (and with fairly little available data). Let's first start by creating a function to retrieve text from a url. 

In [ ]:
def wikipedia_to_paragraphs(url):
    """
    Retrieves a URL from wikipedia, and returns a list of paragraphs 
    (based on the 'p' html paragraph tag) 
    """
    files_by_url = {
      "http://en.wikipedia.org/wiki/Ford_(crossing)": "ford_crossing.txt",
      "http://en.wikipedia.org/wiki/Ford": "ford_car.txt",
      "http://en.wikipedia.org/wiki/Apple": "apple_fruit.txt",
      "http://en.wikipedia.org/wiki/Apple_Inc.": "apple_inc.txt"
    }
    
    try:
        with open("small_data/{}".format(files_by_url[url]), encoding='utf-8') as wiki_file:
            soup = BeautifulSoup(wiki_file.read(), 'lxml')\
            .find(attrs={'id':'mw-content-text'})
    except KeyError:
        soup = BeautifulSoup(urlopen(url), 'lxml').find(attrs={'id':'mw-content-text'})
    
    # The text is littered by references like [n].  Drop them.
    def drop_refs(s):
        return ''.join( re.split('\[\d+\]', s) )
    
    return [drop_refs(p.text) for p in soup.find_all('p') if p.text != '']

We'll then perform feature engineering in a transformer class. 

In [ ]:
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin

class AdHocFeatures(BaseEstimator, TransformerMixin):
    """
    Given a keyword (e.g., "apple"), will transform documents into an
    encoding of several ad hoc features of each occurrences of the keyword:
        - If the keyword is capitalized
        - If it is plural
        - If it is possessive (in the stupid sense of being followed by 's)
        - If the keyword is a verb (e.g., for Ford vs ford)
    """

    def __init__(self, keyword):
        self.keyword = nlp(keyword)[0].lemma_
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return np.asarray([self.transform_doc(x) for x in X])
    
    def feature_posessive(self, doc):
        ## N.B. spaCy will tokenize "Apple's" as ["Apple", "'s"]
        hits = [i for i, word in enumerate(doc) if word.lemma_ == self.keyword]
        return sum((i + 1) < len(doc) and doc[i+1].text == "'s" for i in hits)
    
    def transform_doc(self, row):
        doc = nlp(row)
        words = [word for word in doc if word.lemma_ == self.keyword]
        return [sum(word.is_title for word in words),
                sum(word.tag_ in ('NNS', 'NNPS') for word in words),
                self.feature_posessive(doc),
                sum(word.pos_ == 'VERB' for word in words)]

Next, we'll make our classifier. To do this, we will use a multinomial **Naive Bayes** model. Detailed information on Naive Bayes can be found in the Naive Bayes notebook, but we'll briefly describe it here: 

> The goal is, given a set of observed features $X_1, \ldots, X_p$, to find the label $Y$ with the maximum conditional probability. In other words, we know what our distributions of words ($X$'s) should look like for a given genre ($Y$) from our training data, and we would like to use this information to find the genre of a body of text ($Y$) given its words ($X$'s) for new data. Bayes' theorem gives us a way to compute the latter conditional probability from the former. 

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline, FeatureUnion

def make_classifier(base_word, meaning1, meaning2):
    """
    Given
        - a base word (e.g., "apple", "ford") that can have ambiguous meaning
        - a pair meaning1 = (name1, url1) of a label for the first meaning, and a Wikipedia URL for it
        - a pair meaning2 = ... for the other meaning
    Returns a classifier that predicts the meaning
    """
    name1, url1 = meaning1
    name2, url2 = meaning2
    para1 = wikipedia_to_paragraphs(url1)
    para2 = wikipedia_to_paragraphs(url2)
    minlen = min(len(para1),len(para2))
    if len(para1) == minlen:
        para2 = para2[:minlen]
    else:
        para1 = para1[:minlen]
    
    stop_words_lemma = set(w.lemma_ for w in nlp(' '.join(STOP_WORDS)))
    def tokenize_lemma(text):
        return [w.lemma_ for w in nlp(text)]

    features = FeatureUnion([('stem_vectorizer',
                              TfidfVectorizer(ngram_range=(1,2),
                                              stop_words=stop_words_lemma.union({base_word}),
                                              tokenizer=tokenize_lemma)),
                             ('ad_hoc', AdHocFeatures(base_word))])
    pipe = Pipeline([('features', features),
                     ('classifier', MultinomialNB())])

    # Build the training data
    train_res  = [name1] * len(para1) + [name2] * len(para2)
    
    return pipe.fit(para1 + para2, train_res)

In [ ]:
base_word = "apple"
options = [ ("fruit", "http://en.wikipedia.org/wiki/Apple"),
            ("company", "http://en.wikipedia.org/wiki/Apple_Inc.") ]
print(make_classifier(base_word, *options).predict([
    "I'm baking a pie with my granny smith apples.",
    "I looked up the recipe on my Apple iPhone.",
    "The apple pie recipe is on my desk.",
    "How is Apple's stock doing?",
    "I'm drinking apple juice.",
    "I have three apples.",
    "Steve Jobs is the CEO of apple.",
    "Steve Jobs likes to eat apples."
]))

We can also do this for other classes of text documents. 

In [ ]:
base_word = "ford"
options = [ ("crossing", "http://en.wikipedia.org/wiki/Ford_(crossing)"),
            ("company", "http://en.wikipedia.org/wiki/Ford") ]
print(make_classifier(base_word, *options).predict([
    "I tried to ford the river and my unfortunate oxen died.",
    "Ford makes cars, though their quality is sometimes in dispute.",
    "The Ford Mustang is an iconic automobile.",
    "The river crossing was shallow, but we could not ford it."
]))

### Exercises / Brainstorming for Improvement:

* To speed up the time it takes to train the model, in this example we train the model with entire paragraphs of text and use it to make predictions about the topical content of sentences.  What are some possible problems with this approach?  Modify the above code to train the model on sentences of text rather than paragraphs.  Does the change make a difference?  


* Change the code to use just the ad hoc features. How does this change the results? Why do you think this is?
Same question as 1, but for the tf-idf features.
Change the formation of tf-idf features as follows -- when doing the tf-idf weighting (in the call to fit appearing in `make_ng_stem_vectorizer`) we pass in the paragraphs as separate documents. How do the results change if we pass them in as just two documents?
What ideas do you think could improve the performance of this model?

### Exit Tickets

1. What are some other options for modeling with text data besides bag of words?
1. How would you account for the fact that word meanings change over time?
1. How do stopwords, stemming, and limiting the # features affect variance-bias?

### Spoilers

Some ideas / hints for the exercises:
  - A key problem with the model is the small amount of training data.  At the least, we could follow links from the given Wikipedia articles.  Better would be to find other sources that directly use the words Apple/apple.
  - In this specific case (apple/Apple) we would do better by using a few human created absolute rules _first_: e.g., typos aside -- apple's will always refer to the company and apples to the fruit, so we do not need to run a more complicated learner. 

## Additional NLP topics and resources


Natural language processing is a big field.  We only (really) talked about a few tools and techniques.  Here are some other terms that are relevant:

 - Context free grammars (and probabilistic context free grammars): This is a simple and basic technique for parsing.  
 - [Word2vec](https://code.google.com/p/word2vec/) is a popular tool for creating a vectorized representation of a text corpus. The learned vectors can then be used to identify/predict words similar to a target, or even (weakly) to reason by analogy. For example, vector('Paris') - vector('France') + vector('Italy') results in a vector that is very close to vector('Rome'), and vector('king') - vector('man') + vector('woman') is close to vector('queen').
 
 To use Word2vec in Python (and get the computation speed improvements) look at [Gensim](https://radimrehurek.com/gensim/models/word2vec.html) and [Cython](https://github.com/cython/cython/wiki/Installing). This [Git repo](https://github.com/danielfrg/word2vec) is an alternative way to access the algorithm. You might also use this [Kaggle competition](https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-2-word-vectors) as a reference.  spaCy gives words a `.vector` property based on [a particular](https://spacy.io/docs#token-distributional) Word2vec model. 

*Copyright &copy; 2019 The Data Incubator.  All rights reserved.*